In [109]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns


# Data Import

In [110]:
df_tracks = pd.read_csv('./data/tracks.csv', index_col=0)
df_tracks.head()

,track_id,track_name,artist,album_type,release_date,decade,popularity,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,4BP3uh0hFLFRb5cjsgLqDh,Fortunate Son,Creedence Clearwater Revival,album,1969-11-02,1960,80,0.640,0.663,0,-7.516,1,0.0374,0.008060,0.1520,0.663,132.770,140773,4
1,20OFwXhEXf12DzwXmaV7fj,Bad Moon Rising,Creedence Clearwater Revival,album,1969-08-03,1960,77,0.508,0.774,2,-5.973,1,0.0321,0.000005,0.0630,0.942,178.946,141600,4
2,1qRA5BS78u3gME0loMl9AA,For What It's Worth,Buffalo Springfield,album,1966-12-05,1960,74,0.653,0.519,2,-10.164,1,0.0497,0.020900,0.1010,0.822,98.883,153693,4
3,6YffUZJ2R06kyxyK6onezL,Ring of Fire,Johnny Cash,compilation,1963-08-06,1960,69,0.659,0.585,7,-8.189,1,0.0288,0.000213,0.3480,0.784,104.111,158427,4
4,2gE95JskwQ1pCACTpGe1Db,Down On The Corner,Creedence Clearwater Revival,album,1969-11-02,1960,72,0.858,0.544,0,-9.289,1,0.0751,0.001090,0.0707,0.874,108.217,166280,4


# Data Cleaning

In [111]:
df_tracks.nunique()

track_id            6000
track_name          5268
artist               761
album_type             3
release_date        1346
decade                 6
popularity            61
danceability         610
energy               915
key                   12
loudness            4828
mode                   2
speechiness          745
instrumentalness    2307
liveness            1155
valence              900
tempo               5711
duration_ms         4791
time_signature         4
dtype: int64

## The dataset of 6000 track only has 5268 unique track names. Indicating there is duplicate songs within this dataset that must be removed.

* First create a new dataframe containing all the duplicate track_names. This will be done to further investigate the duplicate track and resolve if possible.
  * Remaining tracks will then be added back to `df_tracks`
* Drop all duplicates from the initial dataframe

In [112]:
df_duplicates = df_tracks[df_tracks['track_name'].duplicated(keep=False)].sort_values('track_name')
df_tracks.drop_duplicates('track_name', keep=False, inplace=True)

df_duplicates.head(7)

,track_id,track_name,artist,album_type,release_date,decade,popularity,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
226,2EaCm5PYjpwuIvRo3ZfEFe,25 Minutes to Go - Live at Folsom State Prison...,Johnny Cash,album,1968-05-01,1960,41,0.596,0.474,3,-15.004,1,0.3320,0.000000,0.1960,0.730,119.555,217667,4
126,6uwERhh3sBcQ9eLcfIBmFV,25 Minutes to Go - Live at Folsom State Prison...,Johnny Cash,album,1968-05-02,1960,46,0.669,0.501,3,-12.511,1,0.2040,0.000002,0.7510,0.854,120.783,176773,4
4973,7ob0221dnx0OOGVSzNZgXO,9 to 5,Dolly Parton,compilation,2005-06-28,2000,47,0.560,0.606,6,-10.866,1,0.0398,0.000006,0.0637,0.803,103.726,180907,4
4804,07j5RLJHwsm4cUb3GGoW3w,9 to 5,Dolly Parton,compilation,2009-10-30,2000,46,0.557,0.615,6,-10.638,1,0.0418,0.000001,0.4870,0.820,104.475,165653,4
2001,4w3tQBXhn5345eUXDGBWZG,9 to 5,Dolly Parton,album,1980,1980,75,0.554,0.783,6,-5.852,1,0.0457,0.000002,0.6310,0.813,105.390,162467,4
4986,4N9ge6XIygbUBExatk2lhc,A Better Man,Clint Black,album,2003-09-22,2000,40,0.653,0.637,4,-10.151,1,0.0275,0.000147,0.1230,0.779,162.177,184160,4
2145,10UIQSUilyh2ttlFeAaaPP,A Better Man,Clint Black,album,1989-05-04,1980,49,0.649,0.667,4,-9.125,1,0.0266,0.000119,0.1170,0.780,162.161,184600,4


## The top 7 records of `df_duplicates` highlights the potential issues that can be encountered.
  * A song that was included at a later date in a compilation album
  * A song that has both album_types of album but release dates years apart
  * A song categorized in the correct decade, but has duplicates. 

### Removing `compilation` album types
* Release year of compilation albums is not reliable and all compilation will be
  removed 
    * This may remove some duplicates that only have an album type of
      compilation but to verify correct date would require manually checking
      release date of those songs.

In [113]:
print(f"There are {len(df_duplicates[df_duplicates['album_type'] == 'compilation'])} in the dataset")

df_duplicates = df_duplicates[df_duplicates['album_type'] != 'compilation']
print(f"The duplicates dataset has {len(df_duplicates)} remaining to address.")

There are 361 in the dataset
The duplicates dataset has 977 remaining to address.


### Removing duplicates `album` album types that have two vastly different release data
 * There are individual decade CSV files that will be cycled through and when a track id is found to be in both that record will be added to a clean dataframe and all records with that title will be dropped from `df_duplicates`
   * If a songs track is found in the 1960 data set that track ID will be kept, all other (i.e. ones with a later date will be dropped)

In [141]:
file_list = ['tracks60s.csv', 'tracks70s.csv', 'tracks80s.csv', 'tracks90s.csv', 'tracks00s.csv', 'tracks10s.csv']
matched = pd.DataFrame()

for file in file_list:
    decade_tracks = pd.read_csv('./data/' + file, index_col=0)
    inner_merge = pd.merge(decade_tracks, df_duplicates, how='inner')
    matched = matched.append(inner_merge)

for name in inner_merge['track_name']:
    df_duplicates = df_duplicates[df_duplicates['track_name'] != name]

df_duplicates.head()

,track_id,track_name,artist,album_type,release_date,decade,popularity,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
226,2EaCm5PYjpwuIvRo3ZfEFe,25 Minutes to Go - Live at Folsom State Prison...,Johnny Cash,album,1968-05-01,1960,41,0.596,0.474,3,-15.004,1,0.3320,0.000000,0.196,0.730,119.555,217667,4
126,6uwERhh3sBcQ9eLcfIBmFV,25 Minutes to Go - Live at Folsom State Prison...,Johnny Cash,album,1968-05-02,1960,46,0.669,0.501,3,-12.511,1,0.2040,0.000002,0.751,0.854,120.783,176773,4
115,4VkgY55sUbfszX3XjS3LxW,A Boy Named Sue - Live at San Quentin State Pr...,Johnny Cash,album,1969-06-05,1960,47,0.685,0.486,10,-11.898,1,0.3230,0.000000,0.716,0.892,101.539,225040,4
23,5xMNjx7uqCzMpJZSo4Wq55,A Boy Named Sue - Live at San Quentin State Pr...,Johnny Cash,album,1969-06-04,1960,59,0.630,0.489,10,-12.144,1,0.1800,0.000000,0.381,0.796,101.876,233813,4
1012,54eZmuggBFJbV7k248bTTt,A Horse with No Name,America,album,1972,1970,76,0.654,0.506,11,-17.180,0,0.0535,0.015500,0.155,0.831,123.179,252240,4


## Lastly the duplicates that remain are duplicated songs that are categorized in the same decade.
* The analysis will only be down to a precision of decade so the duplicates
  remaining will be dropped keep the default of the first value encountered.  

In [145]:
df_duplicates.drop_duplicates(subset=['track_name'], keep='first', inplace=True)
df_duplicates.nunique()

track_id            385
track_name          385
artist              154
album_type            2
release_date        201
decade                4
popularity           53
danceability        264
energy              297
key                  12
loudness            379
mode                  2
speechiness         212
instrumentalness    244
liveness            263
valence             299
tempo               383
duration_ms         377
time_signature        3
dtype: int64

### `df_duplicates` now consists of 385 unique track_id and track_names
* Append `df_duplicates` back to the initial dataframe

In [146]:
df_tracks.append(df_duplicates)
df_tracks.nunique()

track_id            4662
track_name          4662
artist               738
album_type             3
release_date        1197
decade                 6
popularity            59
danceability         595
energy               896
key                   12
loudness            4004
mode                   2
speechiness          704
instrumentalness    1984
liveness            1073
valence              890
tempo               4527
duration_ms         3974
time_signature         4
dtype: int64